## Order Dependent Massey

This notebook contains an order dependent version of Massey’s method. If you want to use Massey’s method, but you want a team’s win against another team to mean something different at the beginning and end of the season, this is the notebook will do that for you. If you want to assume that teams change over the coarse of the season, this is a good notebook for you.

This notebook values point differential of results, while the Order Dependent Colley notebook only values wins, losses, and who those are against

-Grant Harkins

In [ ]:
#This cell just collects filepaths and filenames to read later, and allows you to set your weights

pathGames = '/FILEPATH/'#filepath for game file
pathTeams = '/FILEPATH/' #filepath for team file
gameFilename = '.txt'
teamFilename = '.txt'

#We got our data from masseyratings.com, so reading the files is based on the structure of those files

#setting weights; if you want an unweighted system, set all weights to same value
weightHomeWin = 2
weightAwayWin = 4
weightNeutralWin = 3
segmentWeighting = [2,3]

In [ ]:
import pandas as pd

teamNames = pd.read_csv(pathTeams + teamFilename, header = None)
numTeams = len(teamNames)

In [ ]:
#column 0 of the game file is days since 1/1/000
#column 1 is YYYYMMDD
#column 2 is team 1 id
#column 3 is team 1 homefield (1= home, -1 = away, 0 = neutral
#column 4 team1 score
#column 5 team2 id
#column 6 is team 2 homefield
#column 7 is team 2 score

games = pd.read_csv(pathGames + gameFilename, header = None)
numGames = len(games)
print(numGames)

In [ ]:
import numpy as np
from math import ceil

dayBeforeSeason = games.loc[0,0] - 1
lastDayOfSeason = games.loc[len(games)-1,0]

ratings = np.zeros(numTeams)
pointDiffs = []
tGames = []
oppRatings = []
for i in range(numTeams):
    pointDiffs.append(0)
    tGames.append(0)
    oppRatings.append(0)


numberCorrectPredictions = 0


for i in range(numGames):
    team1ID = games.loc[i,2] - 1
    team1Score = games.loc[i,4]
    team1Loc = games.loc[i,3]

    team2ID = games.loc[i,5] - 1
    team2Score = games.loc[i,7]
    team2Loc = games.loc[i,6]

    currentDay = games.loc[i,0]

    if team1Score > team2Score and ratings[team1ID] > ratings[team2ID]:# and (i >= 300):
        numberCorrectPredictions += 1
    elif team2Score > team1Score and ratings[team2ID] > ratings[team1ID]:# and (i >= 300):
        numberCorrectPredictions += 1
    elif team1Score == team2Score and ratings[team1ID] == ratings[team2ID]: #and (i >= 300):
        numberCorrectPredictions += 1

    numberSegments = len(segmentWeighting)
    weightIndex = ceil(numberSegments*((currentDay-dayBeforeSeason)/(lastDayOfSeason-dayBeforeSeason))) - 1
    timeWeight = segmentWeighting[weightIndex]

    if team1Score > team2Score:  # Team 1 won        
        if (team1Loc == 1):      # Home win
            K = weightHomeWin*timeWeight
        elif (team1Loc == -1):   # Away win
            K = weightAwayWin*timeWeight
        else:                    # Neutral court win
            K = weightNeutralWin*timeWeight
    else:                        # Team 2 won
        if (team2Loc == 1):      # Home win
            K = weightHomeWin*timeWeight
        elif (team2Loc == -1):   # Away win
            K = weightAwayWin*timeWeight
        else:                    # Neutral court win
            K = weightNeutralWin*timeWeight
            
    #fill in the lists
    pointDiffs[team1ID] += K*(team1Score - team2Score)
    pointDiffs[team2ID] += K*(team2Score - team1Score)
    tGames[team1ID] += K*1
    tGames[team2ID] += K*1
    oppRatings[team1ID] += K*ratings[team2ID]
    oppRatings[team2ID] += K*ratings[team1ID]
    
    #update the ratings
    ratings[team1ID] = (pointDiffs[team1ID] + oppRatings[team1ID]) / tGames[team1ID]
    ratings[team2ID] = (pointDiffs[team2ID] + oppRatings[team2ID]) / tGames[team2ID]

## Sorting and Printing Rankings/Ratings

In [ ]:

iSort = np.argsort(-ratings)
k = 0
print('\n\n************** OD Massey Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
if k == 0:
    for i in range(numTeams):
        print(f'{i+1:4d}   {ratings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')
else:
    for i in range(k):
        print(f'{i+1:4d}   {ratings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return

print(f'Predictability: {numberCorrectPredictions/(numGames)*100:.2f}%')